In [ ]:
import numpy as np 
import pandas as pd
import pickle as pkl
import meshio
import torch
from tqdm import tqdm
from CardioMesh.CardiacMesh import Cardiac3DMesh

In [ ]:
def read_point_cloud_from_obj(obj_file):
    with open(obj_file, 'r') as file:
        lines = file.readlines()

    points = []
    for line in lines:
        if line.startswith('v '):
            # Split the line and extract the x, y, and z coordinates
            parts = line.split()
            x = float(parts[1])
            y = float(parts[2])
            z = float(parts[3])
            points.append((x, y, z))

    return points

In [ ]:
# Usage
mine_file = 'final_target_2.obj'
my_atlas = read_point_cloud_from_obj(mine_file)
my_atlas = np.array(my_atlas)

center = my_atlas.mean(0)
scale = max(torch.Tensor(my_atlas).abs().max(0)[0])
my_atlas = my_atlas - center
my_atlas = my_atlas / scale

# print(my_atlas.shape)
print(my_atlas)

In [ ]:
wenjia_verts = np.load("mallas/lv_source_verts.npy", allow_pickle=True)
center = wenjia_verts.mean(0)
scale = max(torch.Tensor(wenjia_verts).abs().max(0)[0])
wenjia_verts = wenjia_verts - center
wenjia_verts = wenjia_verts / scale
wenjia_verts

In [ ]:
# myo = Cardiac3DMesh("/home/rodrigo/01_repos/CardioMesh/data/template/myo_ED_AHA17.vtk")
# aha_mapping = {}
# for i, row in pd.DataFrame(myo.subpartID).iterrows():
#     aha_mapping[i] = row[0]
# pkl.dump(aha_mapping, open("aha_mapping.pkl", "wb"))
aha_mapping = pkl.load(open("aha_mapping.pkl", "rb"))

In [ ]:
mapping = {}
for i, p in tqdm(enumerate(my_atlas)):    
    j = ((wenjia_verts - p)**2).sum(1).argmin().numpy()
    # print(j)
    mapping[i] = aha_mapping[int(j)]

In [ ]:
verts = np.load("/home/rodrigo/02_jpy_drafts/mallas/lv_target_verts.npy", allow_pickle=True)
faces = np.load("/home/rodrigo/02_jpy_drafts/mallas/lv_target_faces.npy", allow_pickle=True)

In [ ]:
meshio.write_points_cells(
  "myatlas_with_aha.vtk",
  verts,
  cells={"triangle": faces},
  point_data={"subpartID": [mapping[i] for i in range(len(mapping)) ]},
)